In [1]:
from typing import Callable, Tuple
from finite_distributions.FiniteDistribution import FiniteDistribution
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sinkhorn.SinkhornRunner as SinkhornRunner
import sinkhorn.SinkhornKernels as skern
import visualizer.joint_distribution_visualizer as jdv
import bisect

In [2]:
def exponential_prob(lamb: float,  x: float) -> float:
    return lamb * np.exp(-lamb * x)

In [3]:
def transport_map(lambda_1: float, lambda_2: float, x: float) -> float:
    return (lambda_1 / lambda_2) * x

In [4]:
lambda_1 = 1.0
lambda_2 = 3.0

In [5]:
xs = [round(x, 5) for x in np.linspace(0, 5, 75)]

In [6]:
s1 = np.sum([exponential_prob(lambda_1, x) for x in xs])
s2 = np.sum([exponential_prob(lambda_2, x) for x in xs])

dist_1 = FiniteDistribution({x: exponential_prob(lambda_1, x) / s1 for x in xs})
dist_2 = FiniteDistribution({x: exponential_prob(lambda_2, x) / s2 for x in xs})

In [ ]:
c = lambda x, y: 0.5*(x - y)**2

In [8]:
# p-norm
ps = [1.1, 1.5, 2., 3., 5.]

SCALE = True

epsilon = 1.
delta = 0.01

outputs = {}
for p in ps:
    sinkhorn_runner = skern.get_pnorm_regularized_runner(p, c)
    print(f"Running p = {p}")
    outputs[p] = sinkhorn_runner.run_sinkhorn(dist_1, dist_2, epsilon / p if SCALE else epsilon, delta)
    print(f"p = {p}, Outer iterations = {outputs[p][-1]}")
# pi_e, f_e, g_e = sinkhorn_runner_entropic.run_sinkhorn(dist_1, dist_2, epsilon, delta)

Running p = 1.1
p = 1.1, Outer iterations = 42
Running p = 1.5
p = 1.5, Outer iterations = 6
Running p = 2.0
p = 2.0, Outer iterations = 3
Running p = 3.0
p = 3.0, Outer iterations = 2
Running p = 5.0
p = 5.0, Outer iterations = 3


In [10]:
fig, axes = plt.subplots(nrows=len(ps), ncols=3, figsize=(12, 20), constrained_layout=True)  # 5 rows, 2 cols
axes = axes.flatten()

for i in range(len(ps)):
    p = ps[i]
    pi = outputs[p][0]

    tick_values = np.arange(0, 6, 1)
    tick_indices = [bisect.bisect_left(xs, val) for val in tick_values]
    x_ticks_tuples = (tick_indices, tick_values)
    y_ticks_tuples = (list(tick_indices), list(reversed(tick_values)))

    conditional_map = pi.calculate_conditional_map()
    conditional_ys = np.array([conditional_map[x] for x in xs])
    
    stdevs = []
    for x in xs:
        conditional_avg = conditional_map[x]
        total_probability = np.sum([pi.get_probability(tup) for tup in (pi.get_keys()) if tup[0] == x])
        marginal_probabilities = np.array([pi.get_probability((x, y)) for y in xs])/total_probability
        variance = np.sum(((xs - conditional_avg)**2) * marginal_probabilities)
        stdev = np.sqrt(variance)
        stdevs.append(stdev)
    stdevs = np.array(stdevs)


    jdv.visualize_joint_probability(pi, support_only_threshold = None, xticks = x_ticks_tuples, yticks = y_ticks_tuples, ax = axes[3 * i], show = False, ylabel_to_set = "", xlabel_to_set="", reverse_sort_y_axis = True)
    axes[3*i].set_ylabel(f"P = {p}, Epsilon = {round(epsilon / p, 3)}")
    jdv.visualize_joint_probability(pi, support_only_threshold = 1e-12,  xticks = x_ticks_tuples, yticks = y_ticks_tuples, ax = axes[3 * i + 1], show = False, ylabel_to_set = "", xlabel_to_set="", reverse_sort_y_axis = True)

    axes[3*i+2].plot(xs, [transport_map(lambda_1, lambda_2, x) for x in xs], color = 'green', linestyle = 'dashed', label = 'Transport Map')
    axes[3*i+2].fill_between(xs, conditional_ys - stdevs, conditional_ys + stdevs, color='blue', alpha=0.2, label='Â±1 Std Dev')
    axes[3*i+2].plot(xs, conditional_ys, color = 'red', label = 'Conditional Map E[Y|X]')
    axes[3*i+2].set_xlim((min(xs), max(xs)))
    axes[3*i+2].set_ylim((min(xs), max(xs)))
    axes[3*i+2].set_xticks(x_ticks_tuples[1])
    axes[3*i+2].set_xticklabels(x_ticks_tuples[1])
    axes[3*i+2].set_yticks(y_ticks_tuples[1])
    axes[3*i+2].set_yticklabels(y_ticks_tuples[1])
    axes[3*i+2].legend()

axes[0].set_title("Joint Density")
axes[1].set_title("Support")
axes[2].set_title("Conditional Map")
plt.suptitle(f"Density and Support Plots for P-Norm Regularized Exponential-on-Exponential Computation. Epsilon{" * p" if SCALE else ""} = {1.0}.")


plt.savefig(f"plots/exp_on_exp_pnorm_ot{"_scaled" if SCALE else ""}.png", dpi = 300)
plt.show()